In [1]:
# здесь только импорт
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
import math
import urllib2
from bs4 import BeautifulSoup
import string
import matplotlib.pyplot as plt
import yfinance as yf
import pandas_datareader.data as web
import copy
import progressbar
import time
import warnings
from datetime import datetime
from multiprocessing import Process
from joblib import Parallel, delayed, wrap_non_picklable_objects
import multiprocessing
from tqdm import tqdm
from tqdm import tqdm_notebook
from joblib import parallel_backend

In [2]:
# примитивная функция для обрубания "слишком" больших значений
# идёт от маленьких к большим, начиная с 15го (значение эмпирическое)
# если следующее значение в 1.25 раза больше, то меняем на меньшее
# если функцию не применять, то выбросы будут мешать другим значениям

def cut_over(series_in):
    New_series = copy.deepcopy(series_in)
    New_series.sort_values(inplace = True, ascending = False)
    for k in reversed(range(300)):
        if 1.2 * New_series.values[k+1] < New_series.values[k]:
            New_series.values[k] = New_series.values[k+1]
    return New_series.sort_index()

In [3]:
# тот же cut_over, только для меньшего числа значений

def cut_over_less(series_in):
    New_series = copy.deepcopy(series_in)
    New_series.sort_values(inplace = True, ascending = False)
    for k in reversed(range(30)):
        if 1.2 * New_series.values[k+1] < New_series.values[k]:
            New_series.values[k] = New_series.values[k+1]
    return New_series.sort_index()

In [4]:
# функция аналогчна cut_over, только обрубает наименькие значения

def cut_under(series_in):
    New_series = copy.deepcopy(series_in)
    New_series.sort_values(inplace = True, ascending = False)
    for k in range(len(series_in)-300, len(series_in)-1):
        if abs(New_series.values[k+1]) > 1.2 * abs(New_series.values[k]):
            New_series.values[k+1] = New_series.values[k]
    return New_series.sort_index()

In [5]:
# функция для расчёта медианы + 3 стандартных отклонения, понадобился для нормализации датасета
def max_value(series_in):
    return np.median(series_in) + 3 * np.std(series_in)

In [6]:
# формируем датасет с сайта http://geolenta.com/. Содержит все тикеры, торгующиеся на спб бирже
# датасет содержит столбец тикеров, столбец имён компаний, столбец отрасли и столбец сектора

print 'Собираем список тикеров, торгуемых на СПб бирже'

dataset = pd.DataFrame()
ticker_to_compare = ''
i = 1
while True:
    table=pd.read_html('http://geolenta.com/?page=' + str(i), header = 0)
    rg_table = table[0]
    dataset_new = rg_table[[rg_table.columns[1], rg_table.columns[2], rg_table.columns[3], rg_table.columns[4]]]
    dataset_new = dataset_new.drop([0])
    if i > 1:
        if dataset_new[dataset_new.columns[0]][1] == ticker_to_compare:
            break
    ticker_to_compare = dataset_new[dataset_new.columns[0]][1]
    dataset = pd.concat([dataset, dataset_new])
    dataset = dataset.reset_index(drop = True)
    i = i + 1
    
tickers_only = dataset[dataset.columns[0]]
print 'Собрано ' + str(len(dataset)) + ' тикеров'


print 'Преобразуем список тикеров'
list_of_names = list()
for i in tqdm_notebook(range(len(dataset))):
    if dataset[dataset.columns[1]][i] <> dataset[dataset.columns[1]][i]:
        dataset[dataset.columns[1]][i] = tickers_only[i]
    my_str = dataset[dataset.columns[1]][i]
    my_str = my_str.replace('M/I Homes, Inc.', 'm-i-homes')
    my_str = my_str.replace(', Inc.', '')
    my_str = my_str.replace(' Inc.', '')
    my_str = my_str.replace(' Group', '')
    my_str = my_str.replace(' Limited', '')
    my_str = my_str.replace(' Corporation', '')
    my_str = my_str.replace(' Company', '')
    my_str = my_str.replace(' plc', '')
    my_str = my_str.replace(' class A', '')
    my_str = my_str.replace(' ClassA', '')
    my_str = my_str.replace(' Class A', '')
    my_str = my_str.replace(' class B', '')
    my_str = my_str.replace(' ClassB', '')
    my_str = my_str.replace(' Class B', '')
    my_str = my_str.replace(' class C', '')
    my_str = my_str.replace(' Class C', '')
    my_str = my_str.replace(' ClassC', '')
    my_str = my_str.replace('The ', '')
    my_str = my_str.replace(' Incorporated', '')
    my_str = my_str.replace(' & Co.', '')
    my_str = my_str.replace(' ADR', '')
    my_str = my_str.replace(' CORP.', '')
    my_str = my_str.replace('.com', '')
    my_str = my_str.replace('.', '')
    my_str = my_str.replace('&', 'and')
    my_str = my_str.replace(' InBev SA/NV', '')
    my_str = my_str.replace('IAC/InterActiveCorp', 'iac-interactivecorp')
    
    my_str = my_str.replace(' ', '-')
    my_str = my_str.replace("'", '')
    my_str = my_str.lower()
    list_of_names.append(my_str)

Собираем список тикеров, торгуемых на СПб бирже
Собрано 1268 тикеров
Преобразуем список тикеров


In [7]:
list_of_names = [x.encode('utf-8') for x in list_of_names]

In [8]:
tickers_only = [x.encode('utf-8') for x in tickers_only]

In [9]:
# заполняем нулями DataFrame

dataset['revenue_growth_10y'] = 0
dataset['revenue_growth_5y'] = 0
dataset['revenue_coef'] = 0
dataset['eps_growth_10y'] = 0
dataset['eps_growth_5y'] = 0
dataset['next_5yr_growth'] = 0
dataset['eps_coef'] = 0
dataset['R_square_10y'] = 0
dataset['years_profitable'] = 0
dataset['years_up'] = 0
dataset['years_strike'] = 0
dataset['market_cap'] = 0
dataset['buyback'] = 0
dataset['ROE'] = 0
dataset['mean_ROE'] = 0
dataset['ROA'] = 0
dataset['mean_ROA'] = 0
dataset['Shareholder_Equity_growth'] = 0
dataset['Profit_margin'] = 0
dataset['Profit_margin_mean'] = 0
dataset['Quick_ratio'] = 0
dataset['Current_ratio'] = 0
dataset['Debt_Equity'] = 0
dataset['payout'] = 0
dataset['payout_5yr'] = 0
dataset['CAGR'] = 0
dataset['max_dd'] = 0
dataset['sharpe'] = 0
dataset['sortino'] = 0
dataset['TTM_div_yield'] = 0
dataset['div_inc_years'] = 0
dataset['div_growth_3yr'] = 0
dataset['div_growth_5yr'] = 0
dataset['div_yield_5yr'] = 0
dataset['curr_stock_price'] = 0
dataset['52_week_high'] = 0
dataset['52_week_low'] = 0

# Блок парсинга значений

In [10]:
# парсим сколько лет подряд повышали дивы/ Годная инфа только на www.dividend.com, но если платят менее 10 лет подряд, то инфы нет
# единственный параметр, который можно намайнить не в цикле

print 'Собираем дивидендную историю с www.dividend.com'

table=pd.read_html('https://www.dividend.com/dividend-stocks/10-year-dividend-increasing-stocks')
rg_table = table[0]
inc_years_table = rg_table[rg_table.columns[[1, 4]]]
inc_years_table.rename(columns={'Stock Symbol': dataset.columns[0]}, inplace=True)
result = pd.merge(dataset, inc_years_table, how="left", on=dataset.columns[0])
dataset['div_inc_years'] = result['No. of Years']

Собираем дивидендную историю с www.dividend.com


C:\Users\user\Anaconda2\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [11]:
def parsing_web_data(i):

    # в стандартном именовании тикеров присутствует дефис перед классом акции
    # сайт macrotrends ест тикеры с точкой вместо дефиса
    
    tickers_only[i] = tickers_only[i].replace('-', '.')

    # набиваем revenue_growth

    try:
        table=pd.read_html('https://www.macrotrends.net/stocks/charts/' + tickers_only[i] + '/' + list_of_names[i] + '/revenue')
        rg_table = table[0]
        rg_table = rg_table.iloc[:, 1].values
        rg_table = [s.replace('$', '') for s in rg_table]
        rg_table = [s.replace(',', '') for s in rg_table]
    except:
        dataset.loc[i, 'revenue_growth_5y'] = None 
        dataset.loc[i, 'revenue_growth_10y'] = None
    
    try:
        x = map(float, rg_table[:rg_table.index('')]) 
        x_rev = np.flip(x)
    except:
        try:
            x = map(float, rg_table) 
            x_rev = np.flip(x)
        except:
            x = None
    
    
    
    try:
        revenue_growth_5y = round(((float(rg_table[0]) / float(rg_table[5])) ** (1./5.) - 1)*100, 2)
        dataset.loc[i, 'revenue_growth_5y'] = revenue_growth_5y
    except:
        dataset.loc[i, 'revenue_growth_5y'] = None        

    try:
        revenue_growth = round(((float(rg_table[0]) / float(rg_table[10])) ** (1./10.) - 1)*100, 2)
        dataset.loc[i, 'revenue_growth_10y'] = revenue_growth
    except:
        try:
            revenue_growth = round(((float(rg_table[0]) / float(rg_table[len(rg_table)-1])) ** (1./(len(rg_table)-1)) - 1)*100, 2)
            dataset.loc[i, 'revenue_growth_10y'] = revenue_growth
        except:
            dataset.loc[i, 'revenue_growth_10y'] = None

    try:
        revenue_change = np.zeros(9)
        for k in range (len(x_rev)-10, len(x_rev)-1):
            revenue_change[k] = (x_rev[k+1] - x_rev[k])/(x_rev[k])*100.
        revenue_coef = round(np.mean(revenue_change) / np.std(revenue_change), 2) 
        dataset.loc[i, 'revenue_coef'] = revenue_coef
    except:
        try:
            revenue_change = np.zeros(len(x_rev)-1)
            for k in range (0, len(x_rev)-1):
                revenue_change[k] = (x_rev[k+1] - x_rev[k])/(x_rev[k])*100.
            revenue_coef = round(np.mean(revenue_change) / np.std(revenue_change), 2) 
            dataset.loc[i, 'revenue_coef'] = revenue_coef
        except:
            dataset.loc[i, 'revenue_coef'] = None

    # набиваем прогнозный eps_growth
    try:
        table=pd.read_html('https://finance.yahoo.com/quote/' + tickers_only[i] +'/analysis?p=' + tickers_only[i])
        rg_table = table[5]

        next_5yr_growth = rg_table[tickers_only[i]][4]
        str_finish = string.find(next_5yr_growth, '%')
        next_5yr_growth = float(next_5yr_growth[:str_finish])
    except:
        next_5yr_growth = 0
    dataset.loc[i, 'next_5yr_growth'] = next_5yr_growth

    # набиваем eps_growth
    try:
        table=pd.read_html('https://www.macrotrends.net/stocks/charts/' + tickers_only[i] + '/' + list_of_names[i] + '/eps-earnings-per-share-diluted')
        rg_table = table[0]
        rg_table = rg_table.iloc[:, 1].values
        rg_table = [s.replace('$', '') for s in rg_table]
        rg_table = [s.replace(',', '') for s in rg_table]
    except:
        dataset.loc[i, 'eps_growth_5y'] = None
    
    try:
        y = rg_table[:rg_table.index('0.00')]
        y = map(float, y) 
    except:
        try:
            y = rg_table
            y = map(float, y) 
        except:
            y = list()
    

    try:
        y_10 = y[:11]
        while y_10[len(y_10)-1] == 0:
            del y[len(y_10)-1]
        X = np.linspace(1, len(y_10), num=len(y_10))
        X = X.reshape(-1, 1)
    except:
        y_10 = y

    try:
        reg = LinearRegression().fit(X, y_10)
        R_square = round(reg.score(X, y_10), 2)
        dataset.loc[i, 'R_square_10y'] = R_square
    except:
        dataset.loc[i, 'R_square_10y'] = None




    y_rev = np.flip(y)

    try:
        eps_change = np.zeros(9)
        for k in range (len(y_rev)-10, len(y_rev)-1):
            eps_change[k] = (y_rev[k+1] - y_rev[k])/(y_rev[k])*100.
        eps_coef = round(np.mean(eps_change) / np.std(eps_change), 2) 
        dataset.loc[i, 'eps_coef'] = eps_coef
    except:
        try:
            eps_change = np.zeros(len(y_rev)-1)
            for k in range (0, len(y_rev)-1):
                eps_change[k] = (y_rev[k+1] - y_rev[k])/(y_rev[k])*100.
            eps_coef = round(np.mean(eps_change) / np.std(eps_change), 2) 
            dataset.loc[i, 'eps_coef'] = eps_coef
        except:
            dataset.loc[i, 'eps_coef'] = None

    years_profitable = 0
    years_up = 0
    years_strike = 0
    for k in range(len(y)):
        if y[k] >= 0:
            years_profitable += 1
    for k in range(len(y) - 1):
        if y[k] > y[k+1]:
            years_up += 1
    for k in range(len(y) - 1):
        if y[k] > y[k+1]:
            years_strike += 1    
        else:
            break
    try:
        years_profitable_rel = float(years_profitable) 
        dataset.loc[i, 'years_profitable'] = years_profitable_rel
    except:
        dataset.loc[i, 'years_profitable'] = None

    try:
        years_up_rel = float(years_up)
        dataset.loc[i, 'years_up'] = years_up_rel
    except:
        dataset.loc[i, 'years_up'] = None

    try:
        years_strike_rel = float(years_strike)
        dataset.loc[i, 'years_strike'] = years_strike_rel
    except:
        dataset.loc[i, 'years_strike'] = None    

    try:
        eps_growth_5y = round((((y[0] / y[5]) ** (1./5.)) - 1)*100, 2)
        dataset.loc[i, 'eps_growth_5y'] = eps_growth_5y
    except:
        try:
            eps_growth_5y = round(((y[0] / y[6]) ** (1./(6.)) - 1)*100, 2)
            dataset.loc[i, 'eps_growth_5y'] = eps_growth_5y
        except:
            try:
                eps_growth_10y = round(((y[0] / y[4]) ** (1./(4.)) - 1)*100, 2)
                dataset.loc[i, 'eps_growth_5y'] = eps_growth_5y
            except:
                dataset.loc[i, 'eps_growth_5y'] = None

    try:
        eps_growth_10y = round(((y[0] / y[10]) ** (1./(10.)) - 1)*100, 2)
        dataset.loc[i, 'eps_growth_10y'] = eps_growth_10y
    except:
        try:
            eps_growth_10y = round(((y[0] / y[11]) ** (1./(11.)) - 1)*100, 2)
            dataset.loc[i, 'eps_growth_10y'] = eps_growth_10y
        except:
            try:
                eps_growth_10y = round(((y[0] / y[9]) ** (1./(9.)) - 1)*100, 2)
                dataset.loc[i, 'eps_growth_10y'] = eps_growth_10y
            except:
                dataset.loc[i, 'eps_growth_10y'] = None

    # набиваем market cap    
    try:
        table=pd.read_html('https://www.macrotrends.net/stocks/charts/' + tickers_only[i] + '/' + list_of_names[i] + '/market-cap')
        market_cap = table[0].iloc[:, 2].values[0]
        market_cap = market_cap.replace('$', '')
        market_cap = float(market_cap.replace('B', ''))
        dataset.loc[i, 'market_cap'] = market_cap
    except:
        dataset.loc[i, 'market_cap'] = None

    # набиваем shares_outstanding_reduction (buyback)
    try:
        table=pd.read_html('https://www.macrotrends.net/stocks/charts/' + tickers_only[i] + '/' + list_of_names[i] + '/shares-outstanding')
        rg_table = table[0]
        rg_table = rg_table.iloc[:, 1].values
        rg_table = map(float, rg_table)
        k = 2
        buyback = round(100 * (rg_table[5] - rg_table[0]) / rg_table[5], 2)
        while math.isnan(buyback):
            buyback = round(100 * (rg_table[5-k] - rg_table[0]) / rg_table[5-k], 2)
            k += 1
        dataset.loc[i, 'buyback'] = buyback
    except:
        dataset.loc[i, 'buyback'] = None


    # набиваем ROE, среднее ROE и Shareholder's Equity    
    try:
        table=pd.read_html('https://www.macrotrends.net/stocks/charts/' + tickers_only[i] + '/' + list_of_names[i] + '/roe')
        rg_table = table[0]
        ROE = rg_table.iloc[:, 3].values
        ROE = [s.replace('%', '') for s in ROE]
        ROE = map(float, ROE)
        ROE_now = ROE[0]
        dataset.loc[i, 'ROE'] = ROE_now

        '''
        j = 0
        ROE_new = list()
        for j in range(len(ROE)):
            if abs(ROE[j]) < 3 * abs(np.mean(ROE)):
                ROE_new.append(ROE[j])
        mean_ROE = round(np.mean(ROE_new), 2)
        '''
        mean_ROE = round(np.median(ROE), 2)
        dataset.loc[i, 'mean_ROE'] = mean_ROE

        Shareholder_Equity = rg_table.iloc[:, 2].values
        Shareholder_Equity = filter(lambda v: v==v, Shareholder_Equity)
        Shareholder_Equity = [s.replace('$', '') for s in Shareholder_Equity]
        Shareholder_Equity = [s.replace('B', '') for s in Shareholder_Equity]
        Shareholder_Equity = map(float, Shareholder_Equity)
        try:
            Shareholder_Equity_growth = round(np.mean(Shareholder_Equity) / np.std(Shareholder_Equity), 2)
        except:
            Shareholder_Equity_growth = 0
        dataset.loc[i, 'Shareholder_Equity_growth'] = Shareholder_Equity_growth
    except:
        dataset.loc[i, 'ROE'] =  None
        dataset.loc[i, 'mean_ROE'] =  None
        dataset.loc[i, 'Shareholder_Equity_growth'] =  None

  # набиваем ROA, среднее ROA
    try:
        table=pd.read_html('https://www.macrotrends.net/stocks/charts/' + tickers_only[i] + '/' + list_of_names[i] + '/roa')
        rg_table = table[0]
        ROA = rg_table.iloc[:, 3].values
        ROA = [s.replace('%', '') for s in ROA]
        ROA = map(float, ROA)
        ROA_now = ROA[0]
        dataset.loc[i, 'ROA'] = ROA_now

        mean_ROA = round(np.median(ROA), 2)
        dataset.loc[i, 'mean_ROA'] = mean_ROA

    except:
        dataset.loc[i, 'ROA'] =  None
        dataset.loc[i, 'mean_ROA'] =  None

    # набиваем net-profit-margin  
    try:
        table=pd.read_html('https://www.macrotrends.net/stocks/charts/' + tickers_only[i] + '/' + list_of_names[i] + '/net-profit-margin')
        rg_table = table[0]
        Profit_margin = rg_table.iloc[:, 3].values
        Profit_margin = [s.replace('%', '') for s in Profit_margin]
        Profit_margin = map(float, Profit_margin)
        Profit_margin_now = Profit_margin[0]
        dataset.loc[i, 'Profit_margin'] = Profit_margin_now
        '''
        try:
            mean_Profit_margin = round(np.mean(Profit_margin[:10]), 2)
        except:
            mean_Profit_margin = round(np.mean(Profit_margin), 2)
        '''
        mean_Profit_margin = round(np.median(Profit_margin), 2)
        dataset.loc[i, 'Profit_margin_mean'] = mean_Profit_margin
    except:
        dataset.loc[i, 'Profit_margin'] =  None
        dataset.loc[i, 'Profit_margin_mean'] =  None


    # набиваем Quick и Current ratios    
    try:
        table=pd.read_html('https://www.macrotrends.net/stocks/charts/' + tickers_only[i] + '/' + list_of_names[i] + '/current-ratio')
        rg_table = table[0]
        trigger = rg_table.iloc[:, 2].values[0]
        Current_ratio = map(float, rg_table.iloc[:, 3].values)
        try:
            trigger = trigger.replace('$', '')
            trigger = float(trigger.replace('B', ''))
            Current_ratio_now = Current_ratio[0]
        except:
            Current_ratio_now = Current_ratio[1]

        dataset.loc[i, 'Current_ratio'] = Current_ratio_now

        table=pd.read_html('https://www.macrotrends.net/stocks/charts/' + tickers_only[i] + '/' + list_of_names[i] + '/quick-ratio')
        rg_table = table[0]
        trigger = rg_table.iloc[:, 2].values[0]
        Quick_ratio = map(float, rg_table.iloc[:, 3].values)
        try:
            trigger = trigger.replace('$', '')
            trigger = float(trigger.replace('B', ''))
            Quick_ratio_now = Quick_ratio[0]
        except:
            Quick_ratio_now = Quick_ratio[1]

        dataset.loc[i, 'Quick_ratio'] = Quick_ratio_now
    except:
        dataset.loc[i, 'Current_ratio'] =  None
        dataset.loc[i, 'Quick_ratio'] =  None


    # набиваем Debt_Equity    
    try:
        table=pd.read_html('https://www.macrotrends.net/stocks/charts/' + tickers_only[i] + '/' + list_of_names[i] + '/debt-equity-ratio')
        rg_table = table[0]
        trigger = rg_table.iloc[:, 2].values[0]
        Debt_Equity = map(float, rg_table.iloc[:, 3].values)
        try:
            trigger = trigger.replace('$', '')
            trigger = float(trigger.replace('B', ''))
            Debt_Equity_now = Debt_Equity[0]
        except:
            Debt_Equity_now = Debt_Equity[1]
        dataset.loc[i, 'Debt_Equity'] = Debt_Equity_now
    except:
        dataset.loc[i, 'Debt_Equity'] =  None


    # для yahoo finance меняем обратно точку на дефис
    tickers_only[i] = tickers_only[i].replace('.', '-')


    # набиваем payout    
    try:
        table=pd.read_html('https://finance.yahoo.com/quote/' + tickers_only[i])
        dataset.loc[i, 'payout'] = round(100 * float(table[1][1][5][:5]) / float(table[1][1][3]), 2)
    except:

        dataset.loc[i, 'payout'] =  0

    # набиваем CAGR, max_dd, sharpe, sortino. Начинаем брать после кризиса 2008, т.к. иначе будет не честно по отношению
    # к компаниям, которые в кризис существовали и претерпели просадку бОльшую, чем новые компании

    try:
        price_history = web.get_data_yahoo(tickers_only[i],'01/01/2009',interval='m')        
        price_history = price_history['Adj Close'].values

        num = len(price_history) / 12.
        CAGR = (price_history[len(price_history)-1] / price_history[0]) ** (1. / num) - 1
        CAGR = round(CAGR * 100, 2)

        daily_change = np.zeros(len(price_history)-1)
        max_drawdown_array = np.zeros((len(price_history)-1, len(price_history)-1))
        for k in range (0, len(price_history)-1):
            daily_change[k] = (price_history[k+1] - price_history[k])/(price_history[k])*100.
            for j in range (k, len(price_history)-1):
                max_drawdown_array[k][j] = (price_history[k] - price_history[j])/(price_history[k])*100.
        max_drawdown = round(max(map(max, max_drawdown_array)), 2)

        sharpe_ratio = round((12**0.5) * np.mean(daily_change) / np.std(daily_change), 2) 

        daily_change_negative = filter(lambda x: x < 0, daily_change)
        sortino_ratio = round((12**0.5) * np.mean(daily_change) / np.std(daily_change_negative), 2) 

        dataset.loc[i, 'CAGR'] = CAGR
        dataset.loc[i, 'max_dd'] = max_drawdown
        dataset.loc[i, 'sharpe'] = sharpe_ratio
        dataset.loc[i, 'sortino'] = sortino_ratio
    except:
        dataset.loc[i, 'CAGR'] = None
        dataset.loc[i, 'max_dd'] = None
        dataset.loc[i, 'sharpe'] = None
        dataset.loc[i, 'sortino'] = None

    # парсим TTM dividend yield

    table=pd.read_html('https://finance.yahoo.com/quote/' + tickers_only[i])
    rg_table = table[1]
    ttm_div_str = rg_table[1][5]
    str_start = string.find(ttm_div_str, '(') + 1
    str_finish = string.find(ttm_div_str, '%')
    try:
        ttm_div_yield = float(ttm_div_str[str_start:str_finish])
    except:
        ttm_div_yield = 0
    dataset.loc[i, 'TTM_div_yield'] = ttm_div_yield

    # для сайта dividendinvestor меняем дефис в тикерах на двойное нижнее подчёркивание
    tickers_only[i] = tickers_only[i].replace('-', '___')
    if tickers_only[i] == 'BRK___A':
        tickers_only[i] = 'BRK__A'
    if tickers_only[i] == 'BRK___B':
        tickers_only[i] = 'BRK__B'   
    if tickers_only[i] == 'LEN___B':
        tickers_only[i] = 'LEN__B'     
    if tickers_only[i] == 'RDS___A':
        tickers_only[i] = 'rdsa'
    if tickers_only[i] == 'RDS___B':
        tickers_only[i] = 'rds__b'


    try:    
        page = urllib2.urlopen('https://www.dividendinvestor.com/dividend-quote/' + tickers_only[i])
        soup = BeautifulSoup(page)
        divs = str(soup)
    except:
        page = urllib2.urlopen('https://www.dividendinvestor.com/dividend-quote/' + tickers_only[i])
        soup = BeautifulSoup(page)
        divs = str(soup)

    # парсим сколько лет подряд повышали дивы (если <10)
    try:
        str_start = string.find(divs, 'Consecutive Dividend Increases')+55
        cons_years_str = divs[str_start:]
        str_start = string.find(cons_years_str, '>')+1
        str_finish = string.find(cons_years_str, 'Years') - 1
        cons_years = float(cons_years_str[str_start:str_finish])
        if dataset['div_inc_years'][i] <> dataset['div_inc_years'][i]:
            if cons_years < 10:
                dataset.loc[i, 'div_inc_years'] = cons_years
            else:
                dataset.loc[i, 'div_inc_years'] = 0
    except:
        dataset.loc[i, 'div_inc_years'] = 0

      
    # парсим 5 year average payout
    try:
        str_start = string.find(divs, 'Dividend Payout Ratio 5 yr Average')+60
        str_finish = string.find(divs, '%', str_start)
        payout_5yr = float(divs[str_start:str_finish].replace(',',''))
        dataset.loc[i, 'payout_5yr'] = payout_5yr
    except:
        dataset.loc[i, 'payout_5yr'] = 0

    # парсим 5 year dividend growth
    try:
        str_start = string.find(divs, 'Dividend Growth Rate 5 yr Average')+59
        str_finish = string.find(divs, '%', str_start)
        div_growth_5yr = float(divs[str_start:str_finish].replace(',',''))
        dataset.loc[i, 'div_growth_5yr'] = div_growth_5yr
    except:
        dataset.loc[i, 'div_growth_5yr'] = 0

    # парсим 3 year dividend growth
    try:
        str_start = string.find(divs, 'Dividend Growth Rate 3 yr Average')+59
        str_finish = string.find(divs, '%', str_start)
        div_growth_3yr = float(divs[str_start:str_finish].replace(',',''))
        dataset.loc[i, 'div_growth_3yr'] = div_growth_3yr
    except:
        dataset.loc[i, 'div_growth_3yr'] = 0

    # парсим Dividend Yield 5 Year Average
    try:
        str_start = string.find(divs, 'Dividend Yield 5 Year Average')+55
        str_finish = string.find(divs, '%', str_start)
        div_yield_5yr = float(divs[str_start:str_finish])
        dataset.loc[i, 'div_yield_5yr'] = div_yield_5yr
    except:
        dataset.loc[i, 'div_yield_5yr'] = 0

    # парсим Latest Close Stock Price
    try:
        str_start = string.find(divs, 'Latest Close Stock Price')+51
        str_finish = string.find(divs, '<', str_start)
        curr_stock_price = float(divs[str_start:str_finish].replace(',',''))
        dataset.loc[i, 'curr_stock_price'] = curr_stock_price
    except:
        dataset.loc[i, 'curr_stock_price'] = 0

    # парсим 52-Week High / Low Stock Price
    try:
        str_start = string.find(divs, '52-Week Low/High Stock Price')+10
        high_price = divs[str_start:]
        str_start = string.find(high_price, '-') + 3
        high_price = high_price[str_start:]
        str_finish = string.find(high_price, '<')
        high_price = float(high_price[:str_finish].replace(',',''))
        dataset.loc[i, '52_week_high'] = high_price
    except:
        dataset.loc[i, '52_week_high'] = 0

    try:
        str_start = string.find(divs, '52-Week Low/High Stock Price')+10
        low_price = divs[str_start:]
        str_start = string.find(low_price, '$') + 1
        low_price = low_price[str_start:]
        str_finish = string.find(low_price, '-') - 1
        low_price = float(low_price[:str_finish].replace(',',''))
        dataset.loc[i, '52_week_low'] = low_price
    except:
        dataset.loc[i, '52_week_low'] = 0   

In [12]:
%%time

warnings.filterwarnings('ignore')

jobs_count = multiprocessing.cpu_count()

print 'Собираем фундаментальные показатели компаний из разных источников'


with parallel_backend('threading', n_jobs = jobs_count):
    Parallel()(delayed(parsing_web_data)(m) for m in tqdm_notebook(range(len(dataset))))


Собираем фундаментальные показатели компаний из разных источников



Wall time: 38min 14s


# Блок заполнения пропусков и ошибок

In [13]:
def fulfil(i):
    
# заполним данные по сортино и просадке, которые считались с ошибкой, из другого API. Ошибка у нас при сортино>3 и max_dd>90

    if dataset_ff['sortino'][i] > 3:
        if dataset_ff['max_dd'][i] > 90:
            tickers_for_ff = dataset_ff[dataset_ff.columns[0]].values[i]
            try:
                panel_data = data.DataReader(tickers_for_ff, 'yahoo', start_date, end_date)
                price_history = panel_data['Adj Close'].values
                daily_change = np.zeros(len(price_history)-1)
                max_drawdown_array = np.zeros((len(price_history)-1, len(price_history)-1))
                for k in range (0, len(price_history)-1):
                    daily_change[k] = (price_history[k+1] - price_history[k])/(price_history[k])*100.
                    for j in range (k, len(price_history)-1):
                        max_drawdown_array[k][j] = (price_history[k] - price_history[j])/(price_history[k])*100.
                max_drawdown = round(max(map(max, max_drawdown_array)), 2)
                dataset_ff['max_dd'][i] = max_drawdown


                daily_change_negative = filter(lambda x: x < 0, daily_change)
                sortino_ratio = round((12**0.5) * np.mean(daily_change) / np.std(daily_change_negative), 2) 
                dataset_ff['sortino'][i] = sortino_ratio
            except: 
                dataset_ff['max_dd'][i] = 100
                dataset_ff['sortino'][i] = 10
                
# ошибка всё ещё может остаться, заменяем ошибочные сортино на минимум, max_dd на 100

    if dataset_ff['sortino'][i] > 3:
        if dataset_ff['max_dd'][i] > 90:
            dataset_ff['sortino'][i] = np.min(dataset_ff['sortino'].values)
            dataset_ff['max_dd'][i] = 100
            
    
# меняем отрицательные ROE на нулевые, а если ROE по модулю превышает удвоенный средний ROE, то переписываем средний в текущий
    if abs(dataset_ff['ROE'][i]) > 2 * abs(dataset_ff['mean_ROE'][i]):
        dataset_ff['ROE'][i] = dataset_ff['mean_ROE'][i]
    if dataset_ff['ROE'][i] < 0:
        dataset_ff['ROE'][i] = 0
# меняем отрицательные ROA на нулевые, а если ROA по модулю превышает удвоенный средний ROA, то переписываем средний в текущий
    if abs(dataset_ff['ROA'][i]) > 2 * abs(dataset_ff['mean_ROA'][i]):
        dataset_ff['ROA'][i] = dataset_ff['mean_ROA'][i]
    if dataset_ff['ROA'][i] < 0:
        dataset_ff['ROA'][i] = 0
# меняем profit margin меньшие -100 на -100, а если profit margin по модулю превышает удвоенный средний profit margin, то переписываем средний в текущий
    if abs(dataset_ff['Profit_margin'][i]) > 2 * abs(dataset_ff['Profit_margin_mean'][i]):
        dataset_ff['Profit_margin'][i] = dataset_ff['Profit_margin_mean'][i]
    if dataset_ff['Profit_margin'][i] < -100:
        dataset_ff['Profit_margin'][i] = -100

In [14]:
%%time

warnings.filterwarnings('ignore')

print 'Пора обработать пропуски в данных'

dataset_ff = copy.deepcopy(dataset)

now = datetime.now()
start_date = '2009-01-01'
end_date = str(now.date())

# меняем бесконечные значения на nan
dataset_ff = dataset_ff.replace([np.inf, -np.inf], np.nan)

# убираем пропуски в Debt_Equity (почему-то один вылезает постоянно)
dataset_ff['Debt_Equity'] = dataset_ff['Debt_Equity'].replace(np.nan, max(dataset_ff['Debt_Equity']))    


# заполняем пропуски в первых 22 колонках минимумами из этих колонок (до Debt/equity, не включая последний)
for col in tqdm_notebook(list(dataset)[4:26]):
    for i in range(len(dataset_ff)):
        if dataset_ff[col][i] <> dataset_ff[col][i]:
            dataset_ff[col][i] = np.min(dataset_ff[col].dropna().values)

            
print 'Исправляем ошибки в данных'

with parallel_backend('threading', n_jobs = jobs_count):
    Parallel()(delayed(fulfil)(m) for m in tqdm_notebook(range(len(dataset_ff))))

# обнуляем отрицательное повышение дивов, иначе при подрезании сольётся весь ряд
dataset_ff.loc[dataset_ff['div_growth_3yr'] < 0, 'div_growth_3yr'] = 0        
dataset_ff.loc[dataset_ff['div_growth_5yr'] < 0, 'div_growth_5yr'] = 0        

# если в div_inc_years пропуск, то эта компания не повышала дивиденды -> присваиваем значение нулю
dataset_ff['div_inc_years'] = dataset_ff['div_inc_years'].replace(np.nan, 0)
        
# если байбек меньше -100, то меняем на -100
dataset_ff.loc[dataset_ff['buyback'] < -100, 'buyback'] = -100
        
# отрицательный payout и debt/equity меняем на максимальный в колонках, т.к. этот столбец мы в итоге будем вычитать
dataset_ff.loc[dataset_ff['payout'] < 0, 'payout'] = np.max(dataset_ff['payout'].dropna().values) 
dataset_ff.loc[dataset_ff['Debt_Equity'] < 0, 'Debt_Equity'] = np.max(dataset_ff['Debt_Equity'].dropna().values)        
        
# пропуски в payout и max_dd меняем на максмумы в колонках
dataset_ff['max_dd'] = dataset_ff['max_dd'].replace(np.nan, np.max(dataset_ff['max_dd'].dropna().values) )
dataset_ff['payout'] = dataset_ff['payout'].replace(np.nan, np.max(dataset_ff['payout'].dropna().values) )
 
# пропуски в CAGR, sharpe и sortino меняем на минимумы
dataset_ff['sortino'] = dataset_ff['sortino'].replace(np.nan, np.min(dataset_ff['sortino'].dropna().values) )
dataset_ff['sharpe'] = dataset_ff['sharpe'].replace(np.nan, np.min(dataset_ff['sharpe'].dropna().values) )
dataset_ff['CAGR'] = dataset_ff['CAGR'].replace(np.nan, np.min(dataset_ff['CAGR'].dropna().values) )
    

# если компания повышает дивы менее 5(3) лет, то и рост дивов за последние 5(3) лет должен быть равным 0        
dataset_ff.loc[dataset_ff['div_inc_years'] < 3, 'div_growth_3yr'] = 0
dataset_ff.loc[dataset_ff['div_inc_years'] < 5, 'div_growth_5yr'] = 0

# Нулевые Quick_ratio меняем на Current_ratio
dataset_ff.loc[dataset_ff['Quick_ratio'] == 0, 'Quick_ratio'] = dataset_ff['Current_ratio']

# применяем функцию cut_over для наиболее проблемных столбцов, где максимальные значения являются неоправданными выбросами
# после всей обработки обязательно снова взглянуть на данные визуально
for col in list(dataset_ff)[4:38]:
    if col == 'div_inc_years':
        continue
    if col == 'div_growth_3yr':
        dataset_ff[col] = cut_over_less(dataset_ff[col])
        continue
    if col == 'div_growth_5yr':
        dataset_ff[col] = cut_over_less(dataset_ff[col])
        continue
    if col == 'years_profitable':
        continue
    if col == 'years_up':
        continue
    if col == 'years_strike':
        continue
    else:
        dataset_ff[col] = cut_over(dataset_ff[col])
        dataset_ff[col] = cut_under(dataset_ff[col])
        
print 'Ошибки удалены, данные очищены'

Пора обработать пропуски в данных



Исправляем ошибки в данных



Ошибки удалены, данные очищены
Wall time: 2min 2s


In [15]:
# удаляем признаки, относящиеся к цене, их потом возьмём в отдельный датасет
dataset_ff = dataset_ff.drop(dataset_ff.columns[[37, 38, 39, 40]], axis='columns')

# Блок простой визуализации с целью обнаружения аномалий

In [ ]:
dataset_ff.boxplot(figsize=(2*dataset_ff.shape[1], 10))

# Блок парсинга данных для скидок

In [43]:
print 'Собираем историю цен'

yf.pdr_override() # <== that's all it takes :-)

for i in range(len(tickers_only)):
    if tickers_only[i] == 'BRK__A':
        tickers_only[i] = 'BRK-A'
    if tickers_only[i] == 'BRK__B':
        tickers_only[i] = 'BRK-B'   
    if tickers_only[i] == 'LEN__B':
        tickers_only[i] = 'LEN-B'     
    if tickers_only[i] == 'rdsa':
        tickers_only[i] = 'RDS-A'
    if tickers_only[i] == 'BF___B':
        tickers_only[i] = 'BF-B'

# download dataframe using pandas_datareader
data_price_history = web.get_data_yahoo(list(tickers_only), start="2019-01-01", end = end_date, auto_adjust = True)

Собираем историю цен
[*********************100%***********************]  1268 of 1268 completed

1 Failed download:
- RDSA: No data found, symbol may be delisted


In [47]:
def parsing_discount(i):
    
     
    try:
        table=pd.read_html('https://finance.yahoo.com/quote/' + tickers_only[i])
        rg_table = table[1]
    except:
        p_e_current = 0
        target_1yr = 0

    try:
        p_e_current = float(rg_table[1][2])
    except:
        p_e_current = 0
    try:
        target_1yr = float(rg_table[1][7])
    except:
        target_1yr = 0 

    try:
        table=pd.read_html('https://finance.yahoo.com/quote/' + tickers_only[i] +'/analysis?p=' + tickers_only[i])
        rg_table = table[5]

        next_5yr_growth = rg_table[tickers_only[i]][4]
        str_finish = string.find(next_5yr_growth, '%')
        next_5yr_growth = float(next_5yr_growth[:str_finish])
    except:
        next_5yr_growth = 0
    try:    
        table=pd.read_html('https://www.macrotrends.net/stocks/charts/' + tickers_only[i] + '/' + list_of_names[i]  + '/pe-ratio')
        rg_table = table[0]
        p_e_history = rg_table.iloc[:, 3].values
        p_e_history = p_e_history[p_e_history != 0]
        p_e_mean = np.median(p_e_history[:10])
    except:
        p_e_mean = 0
    
    try:
        price_history =  data_price_history['Close'][data_price_history['Close'].columns[i]].values
        m1 = np.histogram(price_history)
        price_now = price_history[len(price_history)-1]
        rem = m1[1][0]
        stop = 0
        price_cluster = list()
        for k in range(len(m1[0])):
            if m1[0][len(m1[0]) - k - 1] > 40:
                price_cluster.append((m1[1][len(m1[0]) - k]))
                stop = 1
            else:
                if stop == 1:
                    price_cluster.append((m1[1][len(m1[0]) - k]))
                    break
        price_cluster_low = price_cluster[len(price_cluster) - 1]
        price_cluster_high = price_cluster[0]
        price_cluster_mid = (price_cluster_low + price_cluster_high) / 2
        sale_cluster = 100 * (price_cluster_mid - price_now) / price_cluster_mid
        
    except:
        sale_cluster = -100
       
    
    dataset_discount.loc[i, 'p_e_current'] = p_e_current
    dataset_discount.loc[i, 'target_1yr'] = target_1yr
    dataset_discount.loc[i, 'next_5yr_growth'] = next_5yr_growth
    dataset_discount.loc[i, 'p_e_mean'] = p_e_mean
    dataset_discount.loc[i, 'sale_cluster'] = sale_cluster
   


In [48]:
%%time

print 'Собираем таблицу скидок'

warnings.filterwarnings('ignore')

dataset_discount = dataset_ff[list(dataset_ff)[0:4]]
dataset_discount['div_yield_5yr'] = dataset['div_yield_5yr']
dataset_discount['div_inc_years'] = dataset_ff['div_inc_years']
dataset_discount['curr_stock_price'] = dataset['curr_stock_price']
dataset_discount['52_week_high'] = dataset['52_week_high']
dataset_discount['52_week_low'] = dataset['52_week_low']
dataset_discount['eps_growth_5y'] = dataset_ff['eps_growth_5y']


with parallel_backend('threading', n_jobs = jobs_count):
    Parallel()(delayed(parsing_discount)(x) for x in tqdm_notebook(range(len(dataset_discount))))


dataset_discount['TTM_div_yield'] = dataset_ff['TTM_div_yield']
dataset_discount = dataset_discount.fillna(0)

Собираем таблицу скидок



Wall time: 19min 8s


# Блок расчёта баллов

In [50]:
print 'Переводим показатели в баллы'

# забираем dataset_ff
dataset_score_calc = copy.deepcopy(dataset_ff)
dataset_score_calc = dataset_score_calc[dataset_score_calc.columns[:37]]
# Теперь считаем баллы
# Цель - сформировать датасет со значениями от 0 до 10, где 10 - лучший показатель, 0 - худший
# основная логика - отрицательные значения делаем нулевыми, слишком большие значения (большие медианы + 3 стд откл) делаем 10
# остальные значения берём через арктангенс для сглаживания (чтобы прирост больших значений был менее значим, чем прирост небольших)

dataset_score_calc['revenue_growth_10y'] = dataset_ff['revenue_growth_10y'].map(lambda x: 0 if x < 0 else x)
dataset_score_calc['revenue_growth_10y'] = dataset_score_calc['revenue_growth_10y'].map(lambda x: 10 if x > max_value(dataset_score_calc['revenue_growth_10y']) else (10 * np.arctan(x/3) / np.arctan(max_value(dataset_score_calc['revenue_growth_10y']))))
dataset_score_calc['revenue_growth_5y'] = dataset_ff['revenue_growth_5y'].map(lambda x: 0 if x < 0 else x)
dataset_score_calc['revenue_growth_5y'] = dataset_score_calc['revenue_growth_5y'].map(lambda x: 10 if x > max_value(dataset_ff['revenue_growth_5y']) else (10 * np.arctan(x/3) / np.arctan(max_value(dataset_ff['revenue_growth_5y']))))
dataset_score_calc['revenue_coef'] = dataset_ff['revenue_coef'].map(lambda x: 0 if x < 0 else x)
dataset_score_calc['revenue_coef'] = dataset_score_calc['revenue_coef'].map(lambda x: 10 if x > max_value(dataset_ff['revenue_coef']) else (10 * np.arctan(x) / np.arctan(max_value(dataset_ff['revenue_coef']))))
dataset_score_calc['eps_growth_10y'] = dataset_ff['eps_growth_10y'].map(lambda x: 0 if x < 0 else x)
dataset_score_calc['eps_growth_10y'] = dataset_score_calc['eps_growth_10y'].map(lambda x: 10 if x > max_value(dataset_ff['eps_growth_10y']) else (10 * np.arctan(x/3) / np.arctan(max_value(dataset_ff['eps_growth_10y']))))
dataset_score_calc['eps_growth_5y'] = dataset_ff['eps_growth_5y'].map(lambda x: 0 if x < 0 else x)
dataset_score_calc['eps_growth_5y'] = dataset_score_calc['eps_growth_5y'].map(lambda x: 10 if x > max_value(dataset_ff['eps_growth_5y']) else (10 * np.arctan(x/3) / np.arctan(max_value(dataset_ff['eps_growth_5y']))))
dataset_score_calc['next_5yr_growth'] = dataset_ff['next_5yr_growth'].map(lambda x: 0 if x < 0 else x)
dataset_score_calc['next_5yr_growth'] = dataset_score_calc['next_5yr_growth'].map(lambda x: 10 if x > max_value(dataset_ff['next_5yr_growth']) else (10 * np.arctan(x/3) / np.arctan(max_value(dataset_ff['next_5yr_growth']))))
dataset_score_calc['eps_coef'] = dataset_ff['eps_coef'].map(lambda x: 0 if x < 0 else x)
dataset_score_calc['eps_coef'] = dataset_score_calc['eps_coef'].map(lambda x: 10 if x > max_value(dataset_ff['eps_coef']) else (10 * np.arctan(x) / np.arctan(max_value(dataset_ff['eps_coef']))))
dataset_score_calc['R_square_10y'] = 10 * dataset_ff['R_square_10y']
dataset_score_calc['years_profitable'] = 10 * dataset_ff['years_profitable'] / max(dataset_ff['years_profitable'])
dataset_score_calc['years_up'] = 10 * dataset_ff['years_up'] / max(dataset_ff['years_up'])
dataset_score_calc['years_strike'] = 10 * dataset_ff['years_strike'] / max(dataset_ff['years_strike'])
dataset_score_calc['market_cap'] = dataset_ff['market_cap'].map(lambda x: 10 if x > 200 else (x / 20))
dataset_score_calc['buyback'] = dataset_ff['buyback'].map(lambda x: 0 if x < 0 else x)
dataset_score_calc['buyback'] = dataset_score_calc['buyback'].map(lambda x: 10 if x > 35 else x / 3.5)
dataset_score_calc['ROE'] = dataset_ff['ROE'].map(lambda x: 0 if x < 0 else x)
dataset_score_calc['ROE'] = dataset_score_calc['ROE'].map(lambda x: 10 if x > max_value(dataset_ff['ROE']) else (10 * np.arctan(x) / np.arctan(max_value(dataset_ff['ROE']))))
dataset_score_calc['mean_ROE'] = dataset_ff['mean_ROE'].map(lambda x: 0 if x < 0 else x)
dataset_score_calc['mean_ROE'] = dataset_score_calc['mean_ROE'].map(lambda x: 10 if x > max_value(dataset_ff['mean_ROE']) else (10 * np.arctan(x) / np.arctan(max_value(dataset_ff['mean_ROE']))))
dataset_score_calc['ROA'] = dataset_ff['ROA'].map(lambda x: 0 if x < 0 else x)
dataset_score_calc['ROA'] = dataset_score_calc['ROA'].map(lambda x: 10 if x > max_value(dataset_ff['ROA']) else (10 * np.arctan(x) / np.arctan(max_value(dataset_ff['ROA']))))
dataset_score_calc['mean_ROA'] = dataset_ff['mean_ROA'].map(lambda x: 0 if x < 0 else x)
dataset_score_calc['mean_ROA'] = dataset_score_calc['mean_ROA'].map(lambda x: 10 if x > max_value(dataset_ff['mean_ROA']) else (10 * np.arctan(x) / np.arctan(max_value(dataset_ff['mean_ROA']))))
dataset_score_calc['Shareholder_Equity_growth'] = dataset_ff['Shareholder_Equity_growth'].map(lambda x: 0 if x < 0 else x)
dataset_score_calc['Shareholder_Equity_growth'] = dataset_score_calc['Shareholder_Equity_growth'].map(lambda x: 10 if x > 10 else x)
dataset_score_calc['Profit_margin'] = dataset_ff['Profit_margin'].map(lambda x: 0 if x < 0 else x)
dataset_score_calc['Profit_margin'] = dataset_score_calc['Profit_margin'].map(lambda x: 10 if x > max_value(dataset_ff['Profit_margin']) else (10 * np.arctan(x) / np.arctan(max_value(dataset_ff['Profit_margin']))))
dataset_score_calc['Profit_margin_mean'] = dataset_ff['Profit_margin_mean'].map(lambda x: 0 if x < 0 else x)
dataset_score_calc['Profit_margin_mean'] = dataset_score_calc['Profit_margin_mean'].map(lambda x: 10 if x > max_value(dataset_ff['Profit_margin_mean']) else (10 * np.arctan(x) / np.arctan(max_value(dataset_ff['Profit_margin_mean']))))
dataset_score_calc['Quick_ratio'] = dataset_score_calc['Quick_ratio'].map(lambda x: 10 if x > 1 else (10 * np.arctan(x) / np.arctan(1) ))
dataset_score_calc['Current_ratio'] = dataset_score_calc['Current_ratio'].map(lambda x: 10 if x > 1 else (10 * np.arctan(x) / np.arctan(1) ))
dataset_score_calc['Debt_Equity'] = dataset_ff['Debt_Equity'].map(lambda x: 0 if x < 0 else x)
dataset_score_calc['Debt_Equity'] = dataset_score_calc['Debt_Equity'].map(lambda x: 10 if np.arctan(x) > 1 else np.arctan(x))
dataset_score_calc['Debt_Equity'] = 10 - dataset_score_calc['Debt_Equity'] 
dataset_score_calc['payout'] = dataset_score_calc['payout'].map(lambda x: 10 if x > max_value(dataset_ff['payout']) else (10 * x / max_value(dataset_ff['payout'])))
dataset_score_calc['payout'] = 10 - dataset_score_calc['payout'] 
dataset_score_calc['payout_5yr'] = dataset_score_calc['payout_5yr'].map(lambda x: 10 if x > max_value(dataset_ff['payout_5yr']) else (10 * x / max_value(dataset_ff['payout_5yr'])))
dataset_score_calc['payout_5yr'] = 10 - dataset_score_calc['payout_5yr'] 
dataset_score_calc['CAGR'] = dataset_ff['CAGR'].map(lambda x: 0 if x < 0 else x)
dataset_score_calc['CAGR'] = dataset_score_calc['CAGR'].map(lambda x: 10 if x > 30 else x/3 )
dataset_score_calc['max_dd'] = dataset_ff['max_dd'].map(lambda x: 100 if x == 0 else x)
dataset_score_calc['max_dd'] = dataset_score_calc['max_dd'].map(lambda x: 10 if x > 90 else (x - min(dataset_score_calc['max_dd']) )/ 9)
dataset_score_calc['max_dd'] = 10 - dataset_score_calc['max_dd'] 
dataset_score_calc['sharpe'] = dataset_ff['sharpe'].map(lambda x: 0 if x < 0 else x)
dataset_score_calc['sharpe'] = dataset_score_calc['sharpe'].map(lambda x: 10 if x > max_value(dataset_ff['sharpe']) else (10 * np.arctan(x) / np.arctan(max_value(dataset_ff['sharpe']))))
dataset_score_calc['sortino'] = dataset_ff['sortino'].map(lambda x: 0 if x < 0 else x)
dataset_score_calc['sortino'] = dataset_score_calc['sortino'].map(lambda x: 10 if x > max_value(dataset_ff['sortino']) else (10 * np.arctan(x) / np.arctan(max_value(dataset_ff['sortino']))))
dataset_score_calc['TTM_div_yield'] = dataset_ff['TTM_div_yield'].map(lambda x: 0 if x < 0 else x)
dataset_score_calc['TTM_div_yield'] = dataset_score_calc['TTM_div_yield'].map(lambda x: 10 if x > 8 else x/0.8 )
dataset_score_calc['div_inc_years'] = dataset_ff['div_inc_years'].map(lambda x: 10 if x > 50 else x / 5)
dataset_score_calc['div_growth_3yr'] = dataset_ff['div_growth_3yr'].map(lambda x: 10 if x > 30 else x / 3)
dataset_score_calc['div_growth_5yr'] = dataset_ff['div_growth_5yr'].map(lambda x: 10 if x > 30 else x / 3)

Переводим показатели в баллы


# Блок расчёта общего рейтинга

In [51]:
print 'Считаем рейтинг'

dataset_rating  = dataset_score_calc[dataset_score_calc.columns[:4]]
dataset_rating['Growth_score'] = map(int, map(round, dataset_score_calc[dataset_score_calc.columns[[4, 5, 6, 7, 8, 8, 8, 9, 9, 9, 10, 11, 13, 14, 22, 23, 29, 29, 32, 32]]].sum(axis=1) / (len(dataset_score_calc.columns[[4, 5, 6, 7, 8, 8, 8, 9, 9, 9, 10, 11, 13, 14, 22, 23, 29, 29, 32, 32]]) / 10.)))
dataset_rating['Dividend_score'] = map(int, map(round, dataset_score_calc[dataset_score_calc.columns[[27, 28, 33, 33, 33, 33, 34, 34, 34, 35, 36]]].sum(axis=1) / (len(dataset_score_calc.columns[[27, 28, 33, 33, 33, 33, 34, 34, 34, 35, 36]]) / 10.)))
dataset_rating['Stability_score'] = map(int, map(round, dataset_score_calc[dataset_score_calc.columns[[10, 11, 12, 12, 14, 15, 15, 15, 15, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 26, 27, 30, 30, 30, 30,  31, 31, 31, 32, 32, 32, 34, 34, 34, 34, 34]]].sum(axis=1) / (len(dataset_score_calc.columns[[10, 11, 12, 12, 14, 15, 15, 15, 15, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 26, 27, 30, 30, 30, 30,  31, 31, 31, 32, 32, 32, 34, 34, 34, 34, 34]]) / 10.)))
dataset_rating['Overall_score'] = map(int, map(round, dataset_score_calc.sum(axis=1) / 3.3))
dataset_rating['Ready_to_boost'] = map(int, map(round, dataset_score_calc[dataset_score_calc.columns[[4, 6, 6, 9]]].sum(axis=1) / (len(dataset_score_calc.columns[[4, 6, 6, 9]]) / 4.)))

Считаем рейтинг


# Блок расчёта скидок

In [76]:
print 'Считаем скидки'

dataset_discount.loc[dataset_discount['div_inc_years'] < 5, 'div_yield_5yr'] = 0
discount_calc = copy.deepcopy(dataset_discount)
discount_calc = discount_calc[discount_calc.columns[:4]]
discount_calc['dividend_discount'] = 100 * (dataset_discount['TTM_div_yield'] - dataset_discount['div_yield_5yr']) / dataset_discount['TTM_div_yield']
discount_calc.loc[dataset_discount['div_yield_5yr'] == 0, 'dividend_discount'] = None
discount_calc['p_e_discount'] = 100 * (dataset_discount['p_e_mean'] - dataset_discount['p_e_current']) / dataset_discount['p_e_mean']
discount_calc.loc[dataset_discount['p_e_current'] == 0, 'p_e_discount'] = None
discount_calc['target_discount'] = 100 * (dataset_discount['target_1yr'] - dataset_discount['curr_stock_price']) / dataset_discount['target_1yr']
discount_calc['paypack_past'] = np.log(dataset_discount['p_e_current'] * (dataset_discount['eps_growth_5y']/100.) + 1) / np.log(1 + dataset_discount['eps_growth_5y']/100.)
discount_calc.loc[discount_calc['paypack_past'] == 0, 'paypack_past'] = None
discount_calc['paypack_future'] = np.log(dataset_discount['p_e_current'] * (dataset_discount['next_5yr_growth']/100.) + 1) / np.log(1 + dataset_discount['next_5yr_growth']/100.)
discount_calc.loc[discount_calc['paypack_future'] == 0, 'paypack_future'] = None
discount_calc['high_low_discount'] = 100 * ((dataset_discount['52_week_high'] + dataset_discount['52_week_low']) / 2. - dataset_discount['curr_stock_price']) / ((dataset_discount['52_week_high'] + dataset_discount['52_week_low']) / 2.)
discount_calc['sale_cluster'] = dataset_discount['sale_cluster']

Считаем скидки


# Блок отображения покупок

In [77]:
print 'Собираем таблицы с покупками'

to_buy_calc = dataset_rating.merge(discount_calc)

Собираем таблицы с покупками


In [78]:
# Если нужны чисто растущие компании
to_buy_calc_growth = copy.deepcopy(to_buy_calc)
full_growth = to_buy_calc_growth.sort_values(by = 'Growth_score', ascending = False).reset_index(drop=True)
del to_buy_calc_growth['dividend_discount']
to_buy_calc_growth = to_buy_calc_growth.sort_values(by = 'Growth_score', ascending = False)[:50].reset_index(drop=True)
to_buy_calc_growth.loc[to_buy_calc_growth['p_e_discount'] < -5, 'p_e_discount'] = None
to_buy_calc_growth.loc[to_buy_calc_growth['target_discount'] < 0, 'target_discount'] = None
to_buy_calc_growth.loc[to_buy_calc_growth['high_low_discount'] < 0, 'high_low_discount'] = None
to_buy_calc_growth.loc[to_buy_calc_growth['sale_cluster'] < 0, 'sale_cluster'] = None
to_buy_calc_growth.loc[to_buy_calc_growth['paypack_past'] > 13, 'paypack_past'] = None
to_buy_calc_growth.loc[to_buy_calc_growth['paypack_future'] > 13, 'paypack_future'] = None
to_buy_calc_growth = to_buy_calc_growth.dropna().sort_values(by = 'Growth_score', ascending = False)

In [79]:
# Если нужны чисто стабильные компании
to_buy_calc_stable = copy.deepcopy(to_buy_calc)
del to_buy_calc_stable['dividend_discount']
full_stable = to_buy_calc_stable.sort_values(by = 'Stability_score', ascending = False).reset_index(drop=True)
to_buy_calc_stable = to_buy_calc_stable.sort_values(by = 'Stability_score', ascending = False)[:50].reset_index(drop=True)
to_buy_calc_stable.loc[to_buy_calc_stable['p_e_discount'] < -5, 'p_e_discount'] = None
to_buy_calc_stable.loc[to_buy_calc_stable['target_discount'] < -5, 'target_discount'] = None
to_buy_calc_stable.loc[to_buy_calc_stable['high_low_discount'] < -5, 'high_low_discount'] = None
to_buy_calc_stable.loc[to_buy_calc_stable['sale_cluster'] < 0, 'sale_cluster'] = None
to_buy_calc_stable.loc[to_buy_calc_stable['paypack_past'] > 15, 'paypack_past'] = None
to_buy_calc_stable.loc[to_buy_calc_stable['paypack_future'] > 15, 'paypack_future'] = None
to_buy_calc_stable = to_buy_calc_stable.dropna().sort_values(by = 'Stability_score', ascending = False)

In [80]:
# Если нужны растущие стабильные компании
to_buy_calc_growth_stable = copy.deepcopy(to_buy_calc)
del to_buy_calc_growth_stable['dividend_discount']
to_buy_calc_growth_stable['growth_stable'] = (to_buy_calc_growth_stable['Growth_score']-np.mean(to_buy_calc_growth_stable['Growth_score']))/(np.std(to_buy_calc_growth_stable['Growth_score'])) + (to_buy_calc_growth_stable['Stability_score']-np.mean(to_buy_calc_growth_stable['Stability_score']))/ (np.std(to_buy_calc_growth_stable['Stability_score']))
full_growth_stable = to_buy_calc_growth_stable.sort_values(by = 'growth_stable', ascending = False).reset_index(drop=True)
to_buy_calc_growth_stable = to_buy_calc_growth_stable.sort_values(by = 'growth_stable', ascending = False)[:50].reset_index(drop=True)
to_buy_calc_growth_stable.loc[to_buy_calc_growth_stable['sale_cluster'] < 0, 'sale_cluster'] = None
to_buy_calc_growth_stable.loc[to_buy_calc_growth_stable['p_e_discount'] < -5, 'p_e_discount'] = None
to_buy_calc_growth_stable.loc[to_buy_calc_growth_stable['target_discount'] < -5, 'target_discount'] = None
to_buy_calc_growth_stable.loc[to_buy_calc_growth_stable['high_low_discount'] < -5, 'high_low_discount'] = None
to_buy_calc_growth_stable.loc[to_buy_calc_growth_stable['paypack_past'] > 15, 'paypack_past'] = None
to_buy_calc_growth_stable.loc[to_buy_calc_growth_stable['paypack_future'] > 15, 'paypack_future'] = None
to_buy_calc_growth_stable = to_buy_calc_growth_stable.dropna().sort_values(by = 'growth_stable', ascending = False)

In [81]:
# Если нужны растущие дивидендные компании

to_buy_growth_dividend = copy.deepcopy(to_buy_calc)
to_buy_growth_dividend['growth_dividend'] = (to_buy_growth_dividend['Growth_score']-np.mean(to_buy_growth_dividend['Growth_score']))/(np.std(to_buy_growth_dividend['Growth_score'])) + (to_buy_growth_dividend['Dividend_score']-np.mean(to_buy_growth_dividend['Dividend_score']))/ (np.std(to_buy_growth_dividend['Dividend_score']))
full_growth_dividend = to_buy_growth_dividend.sort_values(by = 'growth_dividend', ascending = False).reset_index(drop=True)
to_buy_growth_dividend = to_buy_growth_dividend.sort_values(by = 'growth_dividend', ascending = False)[:100].reset_index(drop=True)
to_buy_growth_dividend.loc[to_buy_growth_dividend['p_e_discount'] < 0, 'p_e_discount'] = None
to_buy_growth_dividend.loc[to_buy_growth_dividend['dividend_discount'] < 0, 'dividend_discount'] = None
to_buy_growth_dividend.loc[to_buy_growth_dividend['sale_cluster'] < 0, 'sale_cluster'] = None
to_buy_growth_dividend.loc[to_buy_growth_dividend['target_discount'] < 0, 'target_discount'] = None
to_buy_growth_dividend.loc[to_buy_growth_dividend['high_low_discount'] < 0, 'high_low_discount'] = None
to_buy_growth_dividend.loc[to_buy_growth_dividend['paypack_past'] > 10, 'paypack_past'] = None
to_buy_growth_dividend.loc[to_buy_growth_dividend['paypack_future'] > 10, 'paypack_future'] = None
to_buy_growth_dividend = to_buy_growth_dividend.dropna().sort_values(by = 'growth_dividend', ascending = False)

In [82]:
# Если нужны сбалансированные компании
to_buy_calc_balanced = copy.deepcopy(to_buy_calc)
full_balanced = to_buy_calc_balanced.sort_values(by = 'Overall_score', ascending = False).reset_index(drop=True)
del to_buy_calc_balanced['dividend_discount']
to_buy_calc_balanced = to_buy_calc_balanced.sort_values(by = 'Overall_score', ascending = False)[:100].reset_index(drop=True)
to_buy_calc_balanced.loc[to_buy_calc_balanced['p_e_discount'] < -5, 'p_e_discount'] = None
to_buy_calc_balanced.loc[to_buy_calc_balanced['target_discount'] < -5, 'target_discount'] = None
to_buy_calc_balanced.loc[to_buy_calc_balanced['sale_cluster'] < 0, 'sale_cluster'] = None
to_buy_calc_balanced.loc[to_buy_calc_balanced['high_low_discount'] < -5, 'high_low_discount'] = None
to_buy_calc_balanced.loc[to_buy_calc_balanced['paypack_past'] > 13, 'paypack_past'] = None
to_buy_calc_balanced.loc[to_buy_calc_balanced['paypack_future'] > 13, 'paypack_future'] = None
to_buy_calc_balanced = to_buy_calc_balanced.dropna().sort_values(by = 'Overall_score', ascending = False)

In [83]:
# Если нужны дивидендные компании
to_buy_dividend = copy.deepcopy(to_buy_calc)
full_dividend = to_buy_dividend.sort_values(by = 'Dividend_score', ascending = False).reset_index(drop=True)
del to_buy_dividend['Growth_score']
del to_buy_dividend['Stability_score']
del to_buy_dividend['Overall_score']
del to_buy_dividend['p_e_discount']
to_buy_dividend['Dividend_yield'] = dataset_ff['TTM_div_yield']
to_buy_dividend = to_buy_dividend.sort_values(by = 'Dividend_score', ascending = False)[:20].reset_index(drop=True)
to_buy_dividend.loc[to_buy_dividend['dividend_discount'] < 0, 'dividend_discount'] = None
to_buy_dividend.loc[to_buy_dividend['high_low_discount'] < 0, 'high_low_discount'] = None
to_buy_dividend.loc[to_buy_dividend['sale_cluster'] < 0, 'sale_cluster'] = None
to_buy_dividend.loc[to_buy_dividend['target_discount'] < 0, 'target_discount'] = None
to_buy_dividend.loc[to_buy_dividend['paypack_past'] > 13, 'paypack_past'] = None
to_buy_dividend.loc[to_buy_dividend['paypack_future'] > 13, 'paypack_future'] = None
to_buy_dividend = to_buy_dividend.dropna().sort_values(by = 'Dividend_score', ascending = False)

In [84]:
# Если нужны дивидендные стабильные компании
to_buy_dividend_stable = copy.deepcopy(to_buy_calc)
to_buy_dividend_stable['Dividend_yield'] = dataset_ff['TTM_div_yield']
to_buy_dividend_stable['dividend_stable'] = (to_buy_dividend_stable['Dividend_score']-np.mean(to_buy_dividend_stable['Dividend_score']))/(np.std(to_buy_dividend_stable['Dividend_score'])) + (to_buy_dividend_stable['Stability_score']-np.mean(to_buy_dividend_stable['Stability_score']))/ (np.std(to_buy_dividend_stable['Stability_score']))
full_dividend_stable = to_buy_dividend_stable.sort_values(by = 'dividend_stable', ascending = False).reset_index(drop=True)
del to_buy_dividend_stable['p_e_discount']
del to_buy_dividend_stable['target_discount']
del to_buy_dividend_stable['Growth_score']
del to_buy_dividend_stable['Overall_score']
del to_buy_dividend_stable['paypack_past']
del to_buy_dividend_stable['paypack_future']
to_buy_dividend_stable = to_buy_dividend_stable.sort_values(by = 'dividend_stable', ascending = False)[:20].reset_index(drop=True)
to_buy_dividend_stable.loc[to_buy_dividend_stable['dividend_discount'] < 0, 'dividend_discount'] = None
to_buy_dividend_stable.loc[to_buy_dividend_stable['sale_cluster'] < 0, 'sale_cluster'] = None
to_buy_dividend_stable.loc[to_buy_dividend_stable['high_low_discount'] < 0, 'high_low_discount'] = None
to_buy_dividend_stable.loc[to_buy_dividend_stable['Dividend_yield'] < dataset_ff[dataset_ff['TTM_div_yield'] != 0]['TTM_div_yield'].mean(), 'Dividend_yield'] = None
to_buy_dividend_stable = to_buy_dividend_stable.dropna().sort_values(by = 'dividend_stable', ascending = False)

In [85]:
# Если нужны компании с потенциальным взрывом роста
to_buy_boost = copy.deepcopy(to_buy_calc)
full_boost = to_buy_boost.sort_values(by = 'Ready_to_boost', ascending = False).reset_index(drop=True)
del to_buy_boost['p_e_discount']
del to_buy_boost['Growth_score']
del to_buy_boost['Overall_score']
to_buy_boost = to_buy_boost.sort_values(by = 'Ready_to_boost', ascending = False)[:50].reset_index(drop=True)

In [86]:
with pd.ExcelWriter('stocks_to_buy.xlsx') as writer:  
    to_buy_calc_growth.to_excel(writer, sheet_name='growth')
    to_buy_calc_stable.to_excel(writer, sheet_name='stable')
    to_buy_dividend.to_excel(writer, sheet_name='dividend')
    to_buy_calc_growth_stable.to_excel(writer, sheet_name='growth_and_stable')
    to_buy_dividend_stable.to_excel(writer, sheet_name='dividend_and_stable')
    to_buy_growth_dividend.to_excel(writer, sheet_name='growth_and_dividend')
    to_buy_calc_balanced.to_excel(writer, sheet_name='balanced')   
    to_buy_boost.to_excel(writer, sheet_name='ready_to_boost')   
    

with pd.ExcelWriter('all_stocks.xlsx') as writer:  
    full_growth.to_excel(writer, sheet_name='growth')
    full_stable.to_excel(writer, sheet_name='stable')
    full_dividend.to_excel(writer, sheet_name='dividend')
    full_growth_stable.to_excel(writer, sheet_name='growth_and_stable')
    full_dividend_stable.to_excel(writer, sheet_name='dividend_and_stable')
    full_growth_dividend.to_excel(writer, sheet_name='growth_and_dividend')
    full_balanced.to_excel(writer, sheet_name='balanced')      
    full_boost.to_excel(writer, sheet_name='ready_to_boost')   
    
print 'Готово!'

Готово!
